In [1]:
import db_communication
import randValueGenerator

In [ ]:
config = {
    'user': 'root',
    'password': 'FDS-apm1',
    'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
    'port': '3306',
    'database': 'Fahrradshop'}
my_db = db_communication.db_communication(config)

In [ ]:
#Reset the DB to an empty DB
my_db.reset_all_tables(True)

In [11]:
#Fill DB with Default Values
import os
from path import Path
import pandas as pd
path = os.getcwd()
path_merkmalcluster = Path(path) / "Bike_DB/merkmalcluster.csv"
path_merkmale = Path(path) / "Bike_DB/merkmale.csv"
path_einzelteile = Path(path) / "Bike_DB/einzelteile.csv"
path_arbeitsschrittgruppe = Path(path) / "Bike_DB/arbeitsschrittgruppe.csv"
path_reihenfolgevorgabe = Path(path) / "Bike_DB/reihenfolgevorgabe.csv"

df_merkmalcluster = pd.read_csv(path_merkmalcluster, sep=',')
df_merkmale = pd.read_csv(path_merkmale, sep=',')
df_einzelteile = pd.read_csv(path_einzelteile, sep=',')
df_arbeitsschrittgruppe = pd.read_csv(path_arbeitsschrittgruppe, sep=',')
df_reihenfolgevorgabe = pd.read_csv(path_reihenfolgevorgabe, sep=',')

df_merkmalcluster.dropna(axis = 1, inplace=True, how = "all")
df_merkmale.dropna(axis = 1, inplace=True, how = "all")
df_einzelteile.dropna(axis = 1, inplace=True, how = "all")
df_arbeitsschrittgruppe.dropna(axis = 1, inplace=True, how = "all")
df_reihenfolgevorgabe.dropna(axis = 1, inplace=True, how = "all")

columns_merkmalcluster = [df_merkmalcluster.columns][0]
columns_merkmale = [df_merkmale.columns][0]
columns_einzelteile = [df_einzelteile.columns][0]
columns_arbeitsschrittgruppe = [df_arbeitsschrittgruppe.columns][0]
columns_reihenfolgevorgabe = [df_reihenfolgevorgabe.columns][0]

column_merkmalcluster = ()
column_merkmale = ()
column_einzelteile = ()
column_arbeitsschrittgruppe = ()
column_reihenfolgevorgabe = ()

for i in range(len(columns_merkmalcluster)):
     column_merkmalcluster += (columns_merkmalcluster[i],)

for i in range(len(columns_merkmale)):
     column_merkmale += (columns_merkmale[i],)

for i in range(len(columns_einzelteile)):
     column_einzelteile += (columns_einzelteile[i],)
        
for i in range(len(columns_arbeitsschrittgruppe)):
     column_arbeitsschrittgruppe += (columns_arbeitsschrittgruppe[i],)
        
for i in range(len(columns_reihenfolgevorgabe)):
     column_reihenfolgevorgabe += (columns_reihenfolgevorgabe[i],)

data_merkmalcluster = df_merkmalcluster.values.tolist()
data_merkmale = df_merkmale.values.tolist()
data_einzelteile = df_einzelteile.values.tolist()
data_arbeitsschrittgruppe = df_arbeitsschrittgruppe.values.tolist()
data_reihenfolgevorgabe = df_reihenfolgevorgabe.values.tolist()

for i in range(0,len(data_merkmalcluster)):
    data_merkmalcluster_as_dict = {}
    for j in range(len(column_merkmalcluster)):
            data_merkmalcluster_as_dict[column_merkmalcluster[j]] = data_merkmalcluster[i][j]
    my_db.insert_data("Merkmalcluster", data_merkmalcluster_as_dict)

for i in range(0,len(data_merkmale)):
    data_merkmale_as_dict = {}
    for j in range(len(column_merkmale)):
            data_merkmale_as_dict[column_merkmale[j]] = data_merkmale[i][j]
    my_db.insert_data("Merkmale", data_merkmale_as_dict)

for i in range(0,len(data_einzelteile)):
    data_einzelteile_as_dict = {}
    for j in range(len(column_einzelteile)):
            data_einzelteile_as_dict[column_einzelteile[j]] = data_einzelteile[i][j]
    my_db.insert_data("Einzelteile", data_einzelteile_as_dict)
    
for i in range(0,len(data_arbeitsschrittgruppe)):
    data_arbeitsschrittgruppe_as_dict = {}
    for j in range(len(column_arbeitsschrittgruppe)):
            data_arbeitsschrittgruppe_as_dict[column_arbeitsschrittgruppe[j]] = data_arbeitsschrittgruppe[i][j]
    my_db.insert_data("Arbeitsschrittgruppe", data_arbeitsschrittgruppe_as_dict)
    
for i in range(0,len(data_reihenfolgevorgabe)):
    ddata_reihenfolgevorgabe_as_dict = {}
    for j in range(len(column_reihenfolgevorgabe)):
            data_reihenfolgevorgabe_as_dict[column_reihenfolgevorgabe[j]] = data_reihenfolgevorgabe[i][j]
    my_db.insert_data("Reihenfolgevorgabe", data_reihenfolgevorgabe_as_dict)

INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('1', 'Rahmen');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('2', 'Federung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('3', 'Lackierung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('4', 'Lenker');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('5', 'Felgen');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('6', 'Radbereifung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('7', 'Gangschaltung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('8', 'Sattel');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('9', 'Bremsen');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES ('10', 'Beleuchtung');
INSERT INTO `Merkmalcluster` (`ClusterNr`, `Merkmalsbezeichnung`) VALUES (

INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('800006', '8', 'Rennradsattel Damen', '164.99');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('800007', '8', 'Sattel Junior', '9.9');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('900001', '9', 'Felgenbremse Standard', '26.9');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('900002', '9', 'Felgenbremse Trekking', '99.99');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('900003', '9', 'Scheibenbremse Standard', '115.9');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('900004', '9', 'Scheibenbremse Rennrad', '349.99');
INSERT INTO `Merkmale` (`MerkmalNr`, `Merkmalcluster`, `Bezeichnung`, `Preis`) VALUES ('1000001', '10', 'Standard Batteriebeleuchtung', '37.5');
INSERT INTO `Merkmale` (`MerkmalNr`, `Mer

INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('6003', 'SCHWALBE Tracer Reifen Active 20 Zoll K-Guard Draht grey stripes-reflex');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('6004', 'Michelin Protek Cross Reifen 28 Zoll Draht Reflex schwarz');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('6005', 'WTB Vigilante Faltreifen 27.5x2.5 Zoll TCS Light Fast Rolling black/light brown');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('6006', 'SCHWALBE HS 180 Drahtreifen 26 Zoll Active Line K-Guard black/brown');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('6007', 'SCHWALBE Pro One Faltreifen 700x25C V-Guard Addix Race black');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('6008', 'Pirelli P Zero Velo TT Faltreifen 28x1.00 Zoll black');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('6009', 'NoTubes Universal Tubeless Ventil 35mm MTB');
INSERT INTO `Einzel

INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('10003', 'Supernova Airstream 2 Frontlicht');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('10004', 'Supernova Airstream Tail Light 2');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('10005', 'Busch + Mueller Lumotec Lyt senso plus Frontscheinwerfer');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('10006', 'Busch + Mueller Birne Dynamo Rücklicht');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('10007', 'Busch + Mueller Lumotec IQ Cyo N plus LED-Scheinwerfer');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('10008', 'Busch + Mueller Toplight Line plus Diodenrücklicht');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('10009', 'MonkeyLink MonkeyLight 70 Lux Recharge Vorne');
INSERT INTO `Einzelteile` (`EinzelteilNr`, `Bezeichnung`) VALUES ('10010', 'MonkeyLink MonkeyLight 70 Lux Recharge Hinten');
INSERT IN

In [5]:
randValueGenerator.fillCustomerTable(5,my_db)

INSERT INTO `Kunde` (`Name`, `Vorname`, `Mail`) VALUES ('Claudia', 'Eslinger', 'claudia.eslinger@apple.com');
INSERT INTO `Kunde` (`Name`, `Vorname`, `Mail`) VALUES ('Jacki', 'Fannin', 'jacki.fannin@web.de');
INSERT INTO `Kunde` (`Name`, `Vorname`, `Mail`) VALUES ('Donya', 'Beeks', 'donya.beeks@apple.com');
INSERT INTO `Kunde` (`Name`, `Vorname`, `Mail`) VALUES ('Cedrick', 'Boll', 'cedrick.boll@gmail.com');
INSERT INTO `Kunde` (`Name`, `Vorname`, `Mail`) VALUES ('Moriah', 'Dobyns', 'moriah.dobyns@web.de');


In [6]:
randValueGenerator.fillConfigAndOrderTable(5,my_db)

INSERT INTO `Auftrag` (`AuftrNr`, `Bestelldatum`, `Lieferdatum`, `KundenNr`) VALUES ('1', '2020-06-06', '2020-06-20', '1');
INSERT INTO `Auftrag` (`AuftrNr`, `Bestelldatum`, `Lieferdatum`, `KundenNr`) VALUES ('2', '2020-04-19', '2020-05-03', '3');
INSERT INTO `Auftrag` (`AuftrNr`, `Bestelldatum`, `Lieferdatum`, `KundenNr`) VALUES ('3', '2020-04-08', '2020-04-22', '4');
INSERT INTO `Auftrag` (`AuftrNr`, `Bestelldatum`, `Lieferdatum`, `KundenNr`) VALUES ('4', '2020-03-03', '2020-03-17', '3');
INSERT INTO `Auftrag` (`AuftrNr`, `Bestelldatum`, `Lieferdatum`, `KundenNr`) VALUES ('5', '2020-02-18', '2020-03-03', '3');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('100', '1', '100004');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('101', '1', '200003');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('102', '1', '300008');
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('103', '1',

In [7]:
my_db.close()

Connection closed
